### Import Data

In [1]:
import pandas as pd
test_df = pd.read_csv('./result/KPM_prompt_based_one_by_one.csv')
print(test_df.columns)
print(test_df)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)', 'confidence_score(1by1)',
       'confidence_score(1by1)(old)'],
      dtype='object')
                                              topic  stance  \
0     Assisted suicide should be a criminal offence       1   
1     Assisted suicide should be a criminal offence       1   
2     Assisted suicide should be a criminal offence       1   
3     Assisted suicide should be a criminal offence       1   
4     Assisted suicide should be a criminal offence       1   
...                                             ...     ...   
7233           The USA is a good country to live in       0   
7234           The USA is a good country to live in       0   
7235           The USA is a good country to live in       0   
7236           The USA is a good country to live in       0   
7237           The USA is a good country to live in       0   

                                           

In [2]:
prediction_df = pd.read_csv('./result/1by1+predictions_in_group.csv')
print(prediction_df.columns)

Index(['topic', 'stance', 'arguments', 'kep_points',
       'predict_kps(Bert_embedding)', 'predict_kps(avg_embedding)',
       'predict_kps(best_embedding)', 'predict_kps(PCA_embedding)'],
      dtype='object')


### Calculate confidence score one by one and get best one

In [3]:
import openai
import yaml
with open("conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
openai.api_key = credentials['environment_variables']['OPENAI_API_KEY']
model_name = "gpt-4"

In [4]:
def generate_and_get_bset(topic, argument, kp):
    try:
        system_prompt = f"""
                You need to evaluate to what extend the 2 sentences matches to each other based on the topic "{topic}" and return a confidence score between 0 and 1. 0 represents not match, 1 represents match.
                """
        rep = openai.ChatCompletion.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": argument+","+kp}
            ]
        )
        result = rep["choices"][0]["message"]["content"]
        float_result = float(result)
        return float_result
    except ValueError:
        print(f"Error converting {result} to float. Regenerating...")
        return generate_and_get_bset(topic, argument, kp)


In [5]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_bert_similarity(sentence1, sentence2):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize and obtain embeddings for each sentence
    tokens1 = tokenizer(sentence1, return_tensors='pt')
    tokens2 = tokenizer(sentence2, return_tensors='pt')

    with torch.no_grad():
        outputs1 = model(**tokens1)
        outputs2 = model(**tokens2)

    # Extract the embeddings from the model outputs
    embedding1 = outputs1.last_hidden_state.mean(dim=1).numpy()
    embedding2 = outputs2.last_hidden_state.mean(dim=1).numpy()

    # Reshape embeddings for cosine similarity calculation
    embedding1 = np.squeeze(embedding1)
    embedding2 = np.squeeze(embedding2)

    # Calculate cosine similarity
    similarity_score = cosine_similarity([embedding1], [embedding2])[0][0]

    return similarity_score

In [12]:
import ast
for index,row in test_df.iloc[5200:5500].iterrows():
    topic = row['topic']
    stance = row['stance']
    argument = row['argument']
    filtered_df = prediction_df[(prediction_df['topic']==topic) & (prediction_df['stance']== stance)]
    # print(filtered_df)
    candidate_kps = filtered_df['predict_kps(PCA_embedding)'].values.tolist()
    candidate_kps = ast.literal_eval(candidate_kps[0])
    print(index)

    best_kp = ''
    confidence_score = 0.0
    for kp in candidate_kps:
        new_score = generate_and_get_bset(topic, argument, kp)
        if new_score >= confidence_score:
            if new_score == confidence_score:
                old_cos = calculate_bert_similarity(argument, best_kp)
                new_cos = calculate_bert_similarity(argument, kp)
                if new_cos>old_cos:
                    best_kp = kp
                    confidence_score = new_score
            else:
                best_kp = kp
                confidence_score = new_score
    print(best_kp)
    print(confidence_score)
    test_df.at[index,'matched_kp(1by1)'] = best_kp
    test_df.at[index,'confidence_score(1by1)'] = confidence_score

5200


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_

Legalizing sex selection could reduce the efforts made by couples to attempt having a child of a specific gender multiple times.
0.95
5201
Legalizing sex selection allows parents, especially those with children of a single gender, to have the freedom of choice in selecting the sex of their next child.
0.9
5202
The right to choose the gender of one's child supports the argument for legalizing sex selection.
1.0
5203


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_

Legalizing sex selection could potentially reduce the rate of abortions based on undesired child gender.
0.2
5204


...

The freedom of speech should allow all forms of expression, including flag burning, unless they cause harm to others.
0.8
5496
The user suggests flag burning as a non-violent form of protest or expression, thus implying it should not be prohibited.
0.9
5497


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_

The freedom of speech should allow all forms of expression, including flag burning, unless they cause harm to others.
0.9
5498


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_

Prohibiting flag burning suppresses a form of expression that causes no physical harm or property damage.
0.7
5499
Prohibiting flag burning suppresses a form of expression that causes no physical harm or property damage.
0.5


In [14]:
test_df.to_csv('./result/KPM_prompt_based_one_by_one.csv', index=False)
print("Add new data successfully!!!!!")

Add new data successfully!!!!!


### Calculate Tokens

In [8]:
import nltk
from nltk.tokenize import word_tokenize

# Download the Punkt tokenizer for word_tokenize
nltk.download('punkt')

def count_tokens(input_text):
    # Tokenize the input text
    tokens = word_tokenize(input_text)

    # Count the number of tokens
    num_tokens = len(tokens)

    return num_tokens, tokens

# Example usage
input_text = "1"+"1"
num_tokens, token_list = count_tokens(input_text)

print(f"Input Text: {input_text}")
print(f"Tokens: {token_list}")
print(f"Number of Tokens: {num_tokens}")

Input Text: 11
Tokens: ['11']
Number of Tokens: 1


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jhluo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import pandas as pd
import ast
token_df = pd.read_csv('./result/KPM_prompt_based_one_by_one.csv')
prediction_df = pd.read_csv('./result/ArgKP21+predictions(v2).csv')
input_text = ''
output_text = ''
for index, row in token_df.iterrows():
    system_prompt = f"""
            You need to evaluate to what extend the 2 sentences matches to each other based on the topic "{row['topic']}" and return a confidence score between 0 and 1. 0 represents not match, 1 represents match.
            """
    filtered_df = prediction_df[(prediction_df['topic']==row['topic']) & (prediction_df['stance']== row['stance'])]
    # print(filtered_df)
    candidate_kps = filtered_df['predict_kps'].values.tolist()
    candidate_kps = ast.literal_eval(candidate_kps[0])
    for kp in candidate_kps:
        user_content = row['argument']+","+kp
        input_text += system_prompt+user_content
        response_content = str(row['confidence_score'])
        output_text += response_content

In [11]:
input_num_tokens, input_token_list = count_tokens(input_text)
output_num_tokens, output_token_list = count_tokens(output_text)
output_num_tokens = output_num_tokens*7238*5
total_cost = input_num_tokens*30/1000000+output_num_tokens*60/1000000

print(f"Number of Input Tokens: {input_num_tokens}")
print(f"Number of Output Tokens: {output_num_tokens}")
print(f"Total cost: {total_cost} $")

Number of Input Tokens: 3023167
Number of Output Tokens: 36190
Total cost: 92.86641 $


### Calcualte average confidence score

In [1]:
import pandas as pd
df = pd.read_csv('./result/KPM_prompt_based_one_by_one.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)',
       'confidence_score(1by1)'],
      dtype='object')


In [2]:
score = df['confidence_score(1by1)'].values.tolist()
print(sum(score)/len(score))

0.8667767339043794
